<a href="https://colab.research.google.com/github/belanatal/PosPUCRio/blob/main/MVPSprintIIb_IsabelaNatal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PUC Rio: Pós Graduação em Ciência de Dados e Analytics**

**MVP Sprint II: Machine Learning & Analytics (40530010056_20230_01)**
**- Parte b**

Aluna: Isabela Fernanda Natal Batista Abreu Gomes

Julho/2023

## Seção I: Introdução
**Classificador de imagens multiclasse - Paisagens Naturais**

**Contexto:** O presente trabalho tem como objetivo criar um modelo de visão computacional baseado em Aprendizado de Máquina Profundo que seja capaz de classificar uma imagem segundo uma das 6 seguintes categorias de paisagens: Prédio; Floresta; Montanha; Geleira; Rua; Mar. Os dados/imagens foram baixados da plataforma Kaggle (Para mais informações "Scene Classification": https://www.kaggle.com/datasets/nitishabharathi/scene-classification).

**Estrutura:** O notebook encontra-se dividido da seguinte forma:

- Importação das bibliotecas
- Acesso e tratamento dos dados que serão a entrada do modelo de deep learning
- Configuração do modelo de deep learning usando uma rede neural convolucional simples com Keras
- Treinamento do modelo de deep learning
- Execução do modelo de deep learning treinado
- Avaliação do modelo de deep learning
- Exportação do modelo de deep learning
- Teste do modelo exportado

### Importando as bibliotecas necessárias para executar o notebook
(Serão utilizadas as bibliotecas pandas e numpy, para a manipulação dos dados; matplotlib, para geração de gráficos; os, para manipulação de pastas e diretórios e bibliotecas voltadas para Machine e Deep Learning, tais como Keras, Tensor Flow e Scikit-Learn)

In [ ]:
!pip install tensorflow==2.13
!pip install keras

In [ ]:
!pip install kaggle

In [ ]:
# Primeiro bloco: Importação das bibliotecas e módulos
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")
#
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.metrics as skm
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from IPython.display import Image, display
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import numpy as np
import matplotlib.pyplot as plt
#Para utilizar o upload e download de arquivos no colab:
from google.colab import drive
from google.colab import files

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/Kaggle_API_Credentials/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download nitishabharathi/scene-classification

 98% 353M/361M [00:07<00:00, 45.5MB/s]
100% 361M/361M [00:07<00:00, 48.0MB/s]


In [ ]:
!unzip scene-classification.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: train-scene classification/train/5499.jpg  
  inflating: train-scene classification/train/55.jpg  
  inflating: train-scene classification/train/550.jpg  
  inflating: train-scene classification/train/5500.jpg  
  inflating: train-scene classification/train/5501.jpg  
  inflating: train-scene classification/train/5502.jpg  
  inflating: train-scene classification/train/5503.jpg  
  inflating: train-scene classification/train/5504.jpg  
  inflating: train-scene classification/train/5505.jpg  
  inflating: train-scene classification/train/5506.jpg  
  inflating: train-scene classification/train/5507.jpg  
  inflating: train-scene classification/train/5508.jpg  
  inflating: train-scene classification/train/5509.jpg  
  inflating: train-scene classification/train/551.jpg  
  inflating: train-scene classification/train/5510.jpg  
  inflating: train-scene classification/train/5511.jpg  
  inflating: train-scene classifi

In [65]:
from google.colab import files
import io
uploaded=files.upload()

Saving validacao.csv to validacao.csv


In [66]:
#arquivo=next(iter(uploaded))
#nome_arquivo=io.StringIO(uploaded[arquivo].decode().strip())

In [56]:
#treino=pd.read_csv(arquivo, sep=",")

In [71]:
treino=pd.read_csv('../content/train.csv',sep=',')
teste=pd.read_csv('../content/test.csv',sep=',')
validacao=pd.read_csv('../content/validacao.csv',sep=',')

In [72]:
treino.head()

,image_name,label
0,0.jpg,Edificio
1,1.jpg,Rua
2,2.jpg,Mar
3,4.jpg,Edificio
4,7.jpg,Rua


In [68]:
# Verificando as dimensões dos DataFrames com as indicações das imagens correspondentes aos conjuntos de treino, teste e validação
print(treino.shape,teste.shape,validacao.shape)

(17034, 2) (4868, 1) (2433, 1)


In [69]:
# Verificando se os DataFrames contêm dados nulos
# Verify if the dataframe don't have NaN or null values
check_for_nan_treino = treino.isnull().values.any()
check_for_nan_teste = teste.isnull().values.any()
check_for_nan_validacao = validacao.isnull().values.any()

print(check_for_nan_treino)
print(check_for_nan_teste)
print(check_for_nan_validacao)

False
False
False


In [70]:
# Criando a lista para armazenar os dados e selecionar a pasta da imagem a ser carregada
data_with_labels = []
labels = []
data_test = []
imagePath = "../content/Imagens/"

## Seção II: Passos iniciais para tratamento dos dados

### Acessando e tratando os dados que serão a entrada do modelo de deep learning

In [ ]:
# Definindo os caminhos dos diretórios para os dados de treinamento, teste e validação
train_dir = 'content/scene/train'
val_dir = 'content/scene/val'
test_dir = 'content/scene/test'

In [ ]:
# Definindo o tamanho do batch e dimensão das imagens
batch_size = 32
img_height = 224
img_width = 224
num_classes = 6

### Preparação dos dados
Será utilizada a classe "ImageDataGenerator", do TensorFlow, destinada a aumentar e pré-processar dados de imagem em tarefas de Aprendizado Profundo.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Criação dos data generators
# Para o subconjunto de treino
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical') #training set
# Para o subconjunto de validação
val_generator = test_datagen.flow_from_directory(val_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical')
# Para o subconjunto de teste
test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical')

## Seção III: Configuração do Modelo de Deep Learning

### Configuração do Modelo usando uma Rede Neural Convolucional (CNN) Simples

Utilizando a biblioteca Keras, será especificada uma camada convolucional 2D com 32 filtros (depois 64, 128 e 256) e função de ativação do tipo ReLU. Na sequência é adicionada uma camada softmax com o mesmo tipo de função de ativação.

In [73]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_height,img_width,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.Conv2D(256, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

NameError: ignored

### Treinamento do modelo de deep learning

Vamos considerar o modelo Adam para otimização. Como métrica para avaliação do modelo, será usado o critério de acurácia. Além disso, por se tratar de um problema de classificação com mais de 2 categorias (6, neste problema), a perda considerada será do tipo entropia cruzada.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Definindo a quantidade de épocas de treinamento do modelo
epochs = 50
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

## Seção IV: Execução do Modelo de Deep Learning
Nesta seção, o modelo treinado será aplicado em cada uma das imagens do dataset de teste, indicando a qual das 6 classes  (Edificio; Floresta; Montanha; Geleira; Rua; Mar) pertence.

In [ ]:
parent_dir = 'content/scene/test'
count_images = 0
class_names = ['Edificio',
               'Floresta',
               'Montanha',
               'Geleira',
               'Rua',
               'Mar']
y_pred = list()
y_true = list()

In [ ]:
# Percorrendo a pasta onde estão salvas as imagens de teste (dividida em diretórios, subdiretórios e arquivos)

for subdir, dirs, files in os.walk(parent_dir):
  for file in files:
    if file.endswith('.png') or file.endswith('.jpg'):  # Considerando que as imagens são de extensão .png ou .jpg
#
      count_images+=1
      split_path = os.path.join(subdir, file).split('/')
      label = split_path[3]
      y_true.append(label)
      img_path = os.path.join(subdir, file)
      display(Image(filename=img_path, width=300))
      img = image.load_img(img_path, target_size=(img_height, img_width))
      figura = image.img_to_array(img)
      figura = np.expand_dims(figura, axis=0)
      figura = figura.astype('float32') / 255.0
#
      # Previsão
      prediction = model.predict(figura)
#
      # Printando as saídas do modelo
      predicted_class = np.argmax(prediction[0])
      probability = prediction[0][predicted_class]
      y_pred.append(class_names[predicted_class])
      print("Categoria Real:", label)
      print("Categoria Prevista:", class_names[predicted_class])
      print("Probabilidade indicada para a categoria prevista:", probability)
      print("\n")

## Seção V: Avaliação do Modelo



In [ ]:
# Calculando as métricas para avaliação do modelo
acuracia = skm.accuracy_score(y_true, y_pred)
precisao = skm.precision_score(y_true, y_pred, average='weighted')
recall = skm.recall_score(y_true, y_pred, average='weighted')
f1score = skm.f1_score(y_true, y_pred, average='weighted')

print("Acurácia: ", acuracia)
print("Precisão: ", precisao)
print("Recall: ", recall)
print("F-Score: ", f1score)

### Matriz de Confusão

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred, labels=class_names)
np.set_printoptions(precisao=2)

plt.figure()
plot_confusion_matrix(cnf_matrix,
                      classes=['Edificio',
                               'Floresta',
                               'Montanha',
                               'Geleira',
                               'Rua',
                               'Mar'],
                      normalize= False,
                      title='Matriz real x predição')

### Exportação do Modelo

In [ ]:
# Vamos salvar o modelo treinado para aplicação futura

# obtendo a data e hora atual
now = datetime.now()

# Definição do formato
format = '%Y-%m-%dT%H%M'

# Converter a data e hora em uma string com o formato especificado
formatted_datetime = now.strftime(format)

path_model = 'content/scene/trained_models'

name_model = 'trained_model_' + formatted_datetime + '.h5'

# salvando o modelo
model.save("%s/%s" % (path_model, name_model))
print("Modelo salvo com o nome: ", name_model)